In [34]:
import pandas as pd
from pymongo import MongoClient
from request_utils import request_llm
from typing import List, Dict
import json
import random
from tqdm import tqdm
import boto3

In [ ]:
_secrets_manager_client = boto3.client("secretsmanager", region_name="eu-west-3")

_secrets = json.loads(
    _secrets_manager_client.get_secret_value(
        SecretId=f"Prod/alloreview"
    )["SecretString"]
)
MONGO_CONNECTION_STRING = (
    "mongodb+srv://alloreview:{}@feedbacksdev.cuwx1.mongodb.net".format(
        _secrets["mongodb"]["password"]
    )
)
mongo_client = MongoClient(MONGO_CONNECTION_STRING)

collection = mongo_client['feedbacks_db']['feedbacks_Prod']

OPENAI_API_KEY = _secrets["openai"]["api_key"]
LLM_API_KEY = _secrets["litellm"]["api_key"]

In [9]:
BRAND = 'gbh_antilles'
MONGO_PASSWORD = "TZ4ejFMVMzInADLP"
TYPE = 'positive'

In [10]:

from_mongo = pd.DataFrame(list(collection.aggregate([
    {
        '$match': {
            'brand': BRAND,
        },
    },
   # { "$sample" : { "size": 2000 } }
])))

from_mongo.shape

(7139, 23)

In [11]:
BRAND_DESCR = '''GBH antilles is a company in the automotive distribution sector.
Feedbacks are collected from the after-sales service (Service après vente) of our car dealerships. 
So "Service après vente" can't be a level1. Because it's the case for all the feedbacks'''

BRAND_DESCR = '''
questions extracted from call with Garance insurance.
These questions are for FAQ generation.
'''

# Generates topics

In [12]:
prompt = """
# Context
My goal is to build a dashboard with some graph for my client : {brand_description}
I extracted some subjects about feedbacks they gave me.
I want to create more profesionnal topics for graph in my dashboard

# Instructions
* I will give you the list of the subjects. Generate the topics in a python list

- A topic can have **two level** : 
Example : "Service client > Réactivité aux demandes"
- A topic have to be **neutral**.
Example of wrong topic : "Service client > Probleme avec les rendez-vous"
Example of good topic: "Service client > Gestions des rendez-vous"
- Name of topics have to be **adapted to the sector**, and looks professional
Example of wrong topic : "Service > comportement"
Example of good topic: "Service client > Qualité et écoute du service client"
- try to not have a lot of unique level1, a **level1 has to contains multiples level2**

# List of subjects:
{subjects}

# Output
Output format should be a json:
{{
    "topics": ["topic1 > xxx", "topic2 > xxx", "topic3 > xxx", ...]
}}
"""


In [14]:
def get_most_occuring_elements(feedback_collection, brand: str, n: int=7000
):
    """
    Get the n most occuring elements in the list
    """

    pipeline = [
        {"$match": {"brand": brand}},
        {"$unwind": "$extractions"},
        {"$match": {"extractions.elementary_subjects": {"$exists": True}}},
        {"$unwind": "$extractions.elementary_subjects"},
        {"$group": {"_id": "$extractions.elementary_subjects", "count": {"$sum": 1}}},
        {"$match": {"count": {"$gt": 4}}},
        {"$sort": {"count": -1}},
        {"$limit": n},
        
    ]

    return list(feedback_collection.aggregate(pipeline)) 

from openai import OpenAI

client_llm = OpenAI(api_key="", base_url="https://llm-api.allobrain.com/")
openaiClient = OpenAI(api_key="")

def request_llm(messages, max_tokens=500, temperature=0, model="claude-3-haiku"):
    res = client_llm.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    return res.choices[0].message.content

In [15]:
e = get_most_occuring_elements(collection, BRAND)

len(e)

391

In [18]:
def parse_json(text: str) -> Dict:
    try:
        # remove text before and after the json
        text = text[text.find('{'):text.rfind('}')+1]
        return json.loads(text)
    except Exception as e:
        print('parse_json() : ', e)
        return {}

def generate_topics(brand_description: str, subjects: List[str],  model='claude-3-5-sonnet'):
    messages = [
        {"role" : "user", "content": prompt.format(brand_description=brand_description, subjects=subjects)}
    ]

    response = ''
    try:
        response = request_llm(messages,model=model, max_tokens=2000)
        res = parse_json(response)

        return res.get('topics', [])
    
    except Exception as e:
        print('generate_topics() : ', e)
        print('response : ', response)
        return []

In [17]:
subjects = (from_mongo['extractions']
            .explode()
            .dropna()
            .apply(lambda x : x.get('elementary_subjects'))
            .dropna()
            .explode()
            .dropna()
            .value_counts()
)

# drop subjects with less than 2 occurences
subjects = subjects[subjects > 5].index.tolist()

print(len(subjects))
subjects

338


["Satisfaction générale de l'accueil",
 'Personnel : Personnel sur place: Accueil agréable',
 'Service : Rapidité du service',
 "Personnel : Qualité de l'écoute : Attention portée aux inquiétudes des clients",
 'Personnel : Personnel sur place : Professionnalisme du personnel',
 'Délais : Respect des délais de remise du véhicule',
 'Politesse du personnel',
 "Services : Temps d'attente : Attente excessive pour la prise en charge du véhicule",
 'Satisfaction générale du client',
 'Satisfaction générale : Expérience globale positive',
 'Service : Qualité de la prise en charge du véhicule',
 "Qualité d'accueil générale",
 "Retour positif sur l'accueil",
 'Service : Efficacité globale du service',
 "Efficacité de l'équipe",
 "Personnel : Qualité de l'écoute : Écoute des clients",
 'Communication : Qualité des explications fournies',
 'Accueil chaleureux',
 'Services : Tarification : Prix élevés',
 'Satisfaction concernant le travail effectué',
 'Personnel : Qualité des conseils',
 'Satisfa

In [22]:
topics = generate_topics(BRAND_DESCR, subjects, model='claude-3-5-sonnet')

topics

["Accueil et Service Client > Qualité générale de l'accueil",
 'Accueil et Service Client > Professionnalisme du personnel',
 'Accueil et Service Client > Réactivité et disponibilité',
 "Accueil et Service Client > Qualité de l'écoute et des conseils",
 'Accueil et Service Client > Communication et information client',
 'Accueil et Service Client > Gestion des rendez-vous',
 'Prestations de Service > Qualité générale du service',
 'Prestations de Service > Rapidité et efficacité des interventions',
 'Prestations de Service > Respect des délais',
 "Prestations de Service > Qualité des réparations et de l'entretien",
 'Prestations de Service > Diagnostic et résolution de problèmes',
 'Prestations de Service > Nettoyage et restitution du véhicule',
 'Gestion des Véhicules > Prise en charge et suivi',
 'Gestion des Véhicules > Véhicule de remplacement',
 'Gestion des Véhicules > État du véhicule après intervention',
 'Gestion des Véhicules > Gestion des problèmes mécaniques récurrents',
 '

Don't hesitate to modify them manually if needed.

In [23]:
# add to topics only the part before the '>'
level1 = [topic.split('>')[0].strip() for topic in topics]
level1 = list(set(level1))
topics = topics + level1

In [24]:
topics

["Accueil et Service Client > Qualité générale de l'accueil",
 'Accueil et Service Client > Professionnalisme du personnel',
 'Accueil et Service Client > Réactivité et disponibilité',
 "Accueil et Service Client > Qualité de l'écoute et des conseils",
 'Accueil et Service Client > Communication et information client',
 'Accueil et Service Client > Gestion des rendez-vous',
 'Prestations de Service > Qualité générale du service',
 'Prestations de Service > Rapidité et efficacité des interventions',
 'Prestations de Service > Respect des délais',
 "Prestations de Service > Qualité des réparations et de l'entretien",
 'Prestations de Service > Diagnostic et résolution de problèmes',
 'Prestations de Service > Nettoyage et restitution du véhicule',
 'Gestion des Véhicules > Prise en charge et suivi',
 'Gestion des Véhicules > Véhicule de remplacement',
 'Gestion des Véhicules > État du véhicule après intervention',
 'Gestion des Véhicules > Gestion des problèmes mécaniques récurrents',
 '

# Mapping

Classify all the elementary_subject into topics.

In [25]:
prompt_classify = '''
# Context
The goal is to **classify** my subject into a specific topic

# List of subject:
{subject}

# List of topics:
{topics}

# Instructions
- Topics are in this format : "level1 > level2"
- You can choose a **full topic** (two levels)  OR **just a level1**.
- If this is not a **perfect match**, put null 

- Start by find the **closest topic**, not the perfect one but all the most relevant topics
- Then find the **perfect one** and **justify** why it's the perfect one.
- If this is not perfect then:
    - Try to find a level1 that is the most relevant and give only the level1 in "topic"
    - Otherwise put null in "topic"
- Give the topic

example of previous classification:
{examples}

# Output
Output format is a json:

{{
    "closed_ones" : "<The most relevant topics are .... because...>",
    "justification" : "<your justification...>",
    "topic" : "<topic>"
}}
'''

In [30]:
def classify_subject(subject: str, topics: List[str], previous_classifications: Dict, model="claude-3-5-sonnet"):
    messages = [
        {"role" : "user", "content": prompt_classify.format(subject=subject, topics=topics, examples=previous_classifications)}
    ]

    response = ''
    try:
        response = request_llm(messages, model=model, max_tokens=1000)
        res = parse_json(response)

        return {
            'elementary_subject' : subject,
            'justification' : res.get('justification', ''),
            'mapping' : res.get('topic', '')
        }
    
    except Exception as e:
        print('classify_subject() : ', e)
        print('response : ', response)
        return {
            'elementary_subject' : subject,
        }

In [31]:
subjects = (from_mongo['extractions']
            .explode()
            .dropna()
            .apply(lambda x : x.get('elementary_subjects'))
            .dropna()
            .explode()
            .dropna()
            .value_counts()
)
print(len(subjects))

important_subjects = subjects[subjects > 3].index.tolist()

all_subjects = subjects.index.tolist()

len(important_subjects)

1205


472

### Example

In [38]:
elementary_subject = random.choice(important_subjects)

print(elementary_subject)

classify_subject(elementary_subject, topics, {}, model="gpt-4o-mini")

Absence de personnel au comptoir


{'elementary_subject': 'Absence de personnel au comptoir',
 'justification': "The perfect match is 'Accueil et Service Client > Réactivité et disponibilité' because the subject specifically addresses the issue of personnel absence, which affects how quickly and effectively customers can be served. This topic encapsulates the core issue of availability in customer service.",
 'mapping': 'Accueil et Service Client > Réactivité et disponibilité'}

1. Classify 20 first elementary_subject into topics with `claude-3-5-sonnet` to have strong and good examples.

2. Then classify the rest of the elementary_subject with `gpt-4o-mini` to finish the mapping.

In [59]:
examples = {}

for subject in important_subjects[:20]:
    res = classify_subject(subject, topics, examples)

    examples[subject] = res
    print(subject, '   --->    ',res.get('mapping'))

examples

La question 'Mon contrat Obsèques est-il toujours valide ?' se rapporte directement à l'assurance obsèques. Parmi les topics proposés, 'Assurance obsèques' est le plus pertinent et correspond parfaitement à la question posée. Cette catégorie traite spécifiquement des contrats d'assurance liés aux obsèques, ce qui inclut naturellement les questions sur la validité de ces contrats. Les autres catégories comme 'Gestion des contrats' ou 'Informations sur les contrats' pourraient être pertinentes, mais 'Assurance obsèques' est plus spécifique et donc plus appropriée pour cette question.
Mon contrat Obsèques est-il toujours valide ?    --->     Assurance obsèques
La question 'Comment augmenter mon assurance habitation chez Garance assurance ?' se rapporte directement à l'assurance habitation. Parmi les topics proposés, 'Assurance habitation' est le plus pertinent et correspond parfaitement à la question posée. Cette catégorie traite spécifiquement des contrats d'assurance habitation, ce qui 

{'Mon contrat Obsèques est-il toujours valide ?': {'elementary_subject': 'Mon contrat Obsèques est-il toujours valide ?',
  'justification': "La question 'Mon contrat Obsèques est-il toujours valide ?' se rapporte directement à l'assurance obsèques. Parmi les topics proposés, 'Assurance obsèques' est le plus pertinent et correspond parfaitement à la question posée. Cette catégorie traite spécifiquement des contrats d'assurance liés aux obsèques, ce qui inclut naturellement les questions sur la validité de ces contrats. Les autres catégories comme 'Gestion des contrats' ou 'Informations sur les contrats' pourraient être pertinentes, mais 'Assurance obsèques' est plus spécifique et donc plus appropriée pour cette question.",
  'mapping': 'Assurance obsèques'},
 'Comment augmenter mon assurance habitation chez Garance assurance ?': {'elementary_subject': 'Comment augmenter mon assurance habitation chez Garance assurance ?',
  'justification': "La question 'Comment augmenter mon assurance 

In [62]:
import concurrent.futures
from tqdm import tqdm

def classify_parallel(
        elementary_subjects: List,
        topics: List,
        previous_classification: str,
        model='gpt-4o-mini',
        chunk_size=20,
):
    ''' 
    Run the classification of the extractions in parallel
        - Create n different clusters depending on the embeddings
        - For each cluster, launch the classification in parallel

    '''
    res = []


    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(
            classify_subject,
            elementary_subject,
            topics,
            previous_classification,
            model
            ) for elementary_subject in elementary_subjects]

        for i in tqdm(range(0, len(futures), chunk_size), desc="Processing chunks"):
            completed_futures, _ = concurrent.futures.wait(futures[i:i+chunk_size], return_when=concurrent.futures.ALL_COMPLETED)

            for future in completed_futures:
                prediction = future.result()
                res.append(prediction)

    
    return res


In [64]:
examples = {key : value['mapping'] for key, value in examples.items()}

Launch the following cell to classify all the elementary_subject into topics.

In [65]:
res = classify_parallel(all_subjects, topics, examples)

Processing chunks: 100%|██████████| 93/93 [04:40<00:00,  3.02s/it]


In [68]:
tmp = pd.DataFrame(res)

tmp.mapping.value_counts()

mapping
Communication avec les clients               237
Paiements et prélèvements                    177
Transmission de documents                    176
Mise à jour des informations personnelles    138
Gestion des contrats                         123
Rentes et versements                         118
Gestion de la retraite                       109
Informations sur les contrats                104
Rachat de contrat                             96
Suivi des dossiers                            86
Accès au compte en ligne                      60
Délais de traitement                          45
Transferts de fonds                           42
Procédures administratives                    39
Fiscalité des contrats                        39
Résiliation de contrat                        39
Documents fiscaux                             34
Rendez-vous et conseil                        30
Accès aux conseillers                         29
Déclaration de décès                          26
Épargne et i